# Task-4 Natural Language Processing(NLP) For Text Generation

# Importing Libraries

In [49]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [50]:
text = """Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: 
once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 
'and what is the use of a book,' thought Alice 'without pictures or conversations?' So she was considering in her own mind 
(as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain 
would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
"""

# Tokenization of Text

In [51]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [52]:
input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [53]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [54]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [55]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Model Training

In [56]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
epochs = 50
history = model.fit(X, y, epochs=epochs, verbose=1)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.0150 - loss: 4.3694
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0649 - loss: 4.3575
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.0889 - loss: 4.3422
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.0760 - loss: 4.3132
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.0714 - loss: 4.2163
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0708 - loss: 4.2431
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0562 - loss: 4.2108
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.0735 - loss: 4.1760
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0579 - loss: 4.1503
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.0662 - loss: 4.0711
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0994 - loss: 4.0676
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.1101 - lo

# Text Generation Function

In [60]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)[0]
        
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                seed_text += " " + word
                break
    return seed_text

# Example Usage

In [62]:
seed_text = "There was a boy named kevin"
generated_text = generate_text(seed_text, 10, max_sequence_len)
print(f"Generated text: {generated_text}")

Generated text: There was a boy named kevin was to get tired tired of sitting by her sister


In [63]:
model.save('text_generation_lstm_model.h5')
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']